In [ ]:
import cv2
import numpy as np
import socket
import pickle
import struct
import time
import io
import PIL.Image
import IPython.display
from pynq.overlays.base import BaseOverlay
from pynq.lib import Button


try:
    base = BaseOverlay("base.bit")
    # Isolate BTN0 for capturing images
    btn0 = base.buttons[0]
except Exception as e:
    print(f"❌ Error initializing PYNQ overlay or buttons: {e}")
    print("Please ensure the bitstream is correct and you are running this on a PYNQ board.")
    exit()



# --- Preprocessing Function ---

def preprocess_for_ocr(image, blur_t=200, contrast_t=255,
                       std_t=40, edge_t=50):
    """
    Applies a series of preprocessing steps to an image to prepare it for OCR.
    """
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Blurriness check
    lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()
    if lap_var < blur_t:
        processed = cv2.bilateralFilter(gray, 9, 15, 15)
    else:
        processed = gray

    # Resize
    h, w = int(processed.shape[0]*3), int(processed.shape[1]*3)
    if h < 3000 and w < 6000:
        processed = cv2.resize(processed, None, fx=3, fy=3, interpolation=cv2.INTER_CUBIC)
    else:
        scale_factor = min(6000 / processed.shape[0], 3000 / processed.shape[1], 3.0)
        new_h, new_w = int(processed.shape[0] * scale_factor), int(processed.shape[1] * scale_factor)
        processed = cv2.resize(processed, (new_w, new_h), interpolation=cv2.INTER_CUBIC)

    # Contrast enhancement
    contrast = np.max(processed) - np.min(processed)
    if contrast < contrast_t:
        clahe = cv2.createCLAHE(clipLimit=1.0, tileGridSize=(18, 18))
        processed = clahe.apply(processed)

    # Thresholding
    std_intensity = np.std(processed)
    if std_intensity < std_t:
        block_size = max(3, (min(processed.shape) // 10) | 1)
        processed = cv2.adaptiveThreshold(processed, 255,
                                          cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                          cv2.THRESH_BINARY,
                                          block_size, 2)

    # Sharpening
    edge_strength = cv2.Canny(processed, 100, 200).sum() / 1000
    if edge_strength < edge_t:
        sharpen_kernel = np.array([[-1, -1, -1],
                                   [-1,  9, -1],
                                   [-1, -1, -1]])
        processed = cv2.filter2D(processed, -1, sharpen_kernel)

    return processed


# --- Helper Function for Smoother Jupyter Video Display ---
def array_to_image_widget(frame, quality=75):
    """
    Converts a NumPy array (frame) to a compressed JPEG IPython Image widget.
    Using JPEG with a quality setting improves display speed over PNG.
    """
    # Convert color space from BGR (OpenCV standard) to RGB for display
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    # Create an in-memory binary stream
    f = io.BytesIO()
    
    # Save the frame to the stream in JPEG format for faster processing
    pil_img = PIL.Image.fromarray(rgb_frame)
    pil_img.save(f, 'jpeg', quality=quality)
    
    # Create an IPython Image object from the binary data
    return IPython.display.Image(data=f.getvalue(), format='jpeg')


# --- Main Application Logic ---

# --- Socket Setup ---
HOST = '192.168.137.1'  # IMPORTANT: Replace with your PC's IP address
PORT = 5002

sock = None
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    print(f"Attempting to connect to Host PC at {HOST}:{PORT}...")
    sock.connect((HOST, PORT))
    print("✅ Connection Successful!")
except socket.error as e:
    print(f"❌ Socket Connection Error: {e}")
    print("Please ensure the PC-side script is running and the IP address is correct.")
    exit()

# --- Camera Setup ---
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("❌ Error: Could not access the camera.")
    if sock:
        sock.close()
    exit()

# --- User Instructions ---
frame_id = 0
print("\n📷 Real-time video is active.")
print("Press BTN0 on the PYNQ board to capture and send a snapshot for OCR.")
print("To stop, interrupt the kernel in Jupyter (Menu > Kernel > Interrupt).")
print("-" * 60)

# This object will hold the display area in Jupyter, allowing for smooth updates
display_handle = None

try:
    while True:
        # --- Capture and Display Real-time Video ---
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Failed to grab a frame from the camera. Retrying...")
            time.sleep(0.5)
            continue

        # Convert frame to an image widget for display
        img_widget = array_to_image_widget(frame, quality=75)
        
        # This logic updates the image in place, preventing flickering
        if display_handle is None:
            display_handle = IPython.display.display(img_widget, display_id=True)
        else:
            display_handle.update(img_widget)
        
        # --- Check for BTN0 Press ---
        if btn0.read() == 1:
            print(f"\nBTN0 pressed! Capturing and processing snapshot...")
            
            # Use the latest frame captured from the camera
            frame_id += 1
            processed_image = preprocess_for_ocr(frame)
            filename = f"frame_{frame_id}_btn0.png"

            # Give visual feedback by showing the processed image
            processed_widget = array_to_image_widget(cv2.cvtColor(processed_image, cv2.COLOR_GRAY2BGR))
            display_handle.update(processed_widget)
            
            # Serialize and send the processed image and filename to the PC
            payload = (filename, processed_image)
            data = pickle.dumps(payload)
            size = struct.pack("Q", len(data))
            sock.sendall(size + data)
            
            print(f"✅ Successfully sent: {filename}")
            
            # Wait for the button to be released to avoid multiple captures
            print("Waiting for button to be released...")
            while btn0.read() == 1:
                time.sleep(0.1)
            print("Button released. Resuming live view.")

        # A short pause to prevent the loop from overwhelming the CPU
        time.sleep(0.03)

except KeyboardInterrupt:
    print("\n🛑 Stream stopped by user (Kernel Interrupt).")
except Exception as e:
    print(f"\nAn unexpected error occurred: {e}")
finally:
    # --- Cleanup ---
    cap.release()
    if sock:
        print("Sending termination signal to host...")
        try:
            # Send a packet of size 0 to signal the end of the stream
            sock.sendall(struct.pack("Q", 0))
            sock.close()
            print("✅ Connection closed and resources released.")
        except socket.error as e:
            print(f"⚠️ Could not send termination signal cleanly: {e}")
    # Clear the final image from the Jupyter output
    IPython.display.clear_output()


In [ ]:
# --- Socket Setup ---
HOST = '192.168.137.1'  # Replace with your PC's IP
PORT = 5002

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

# Open USB camera
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("❌ Error: Could not access the camera.")
    sock.close()
    exit()

frame_id = 0
print("📷 Streaming from camera... Interrupt kernal to stop.")

try:
    while True:
        ret, frame = cap.read()
        if not ret:
            print("⚠️ Failed to grab frame")
            continue

        frame_id += 1
        processed = preprocess_for_ocr(frame)
        filename = f"frame_{frame_id}.png"

        # --- Display processed frame inline ---
        plt.figure(figsize=(6, 4))
        plt.imshow(processed, cmap='gray')
        plt.title(f"Processed Frame {frame_id}")
        plt.axis('off')
        plt.show()

        # --- Serialize and send ---
        payload = (filename, processed)
        data = pickle.dumps(payload)
        size = struct.pack("Q", len(data))
        sock.sendall(size + data)

        print(f"✅ Sent frame {frame_id}")

        # Wait before next capture
        time.sleep(10)

except KeyboardInterrupt:
    print("🛑 Stream stopped by user.")

finally:
    cap.release()
    sock.sendall(struct.pack("Q", 0))  # Termination signal
    sock.close()
    print("✅ Connection closed.")
